In [1]:
#https://www.kaggle.com/code/phamvanvung/shap-for-lstm
#https://charlieoneill11.github.io/charlieoneill/python/lstm/pytorch/2022/01/14/lstm2.html
# https://stackoverflow.com/questions/66080745/keras-model-predicts-nan
# load the general libraries
import pandas as pd
import numpy  as np
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.simplefilter("ignore")

# Making the plots standard 
%matplotlib inline
plt.rcParams["figure.figsize"] = [16, 5]

In [2]:

from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Flatten
import shap

print("Tensorflow version is:", tf.__version__)
print("SHAP version is:", shap.__version__)

Tensorflow version is: 2.15.0
SHAP version is: 0.43.0


In [3]:
#import the training data and test data pickles 
train_df = pd.read_pickle("train_df.pickle")
test_df  = pd.read_pickle("test_df.pickle")

#get the training std 
trainrtt_mean = train_df['last_rtt'].mean()
trainrtt_std = train_df['last_rtt'].std()

In [4]:
# Choose your features and target
cols_dropped = ['date','last_rtt','normalizzed_rtt','src_names']

X_train = train_df.drop(columns=cols_dropped, axis=1)
y_train = train_df['normalizzed_rtt']

X_test = test_df.drop(columns=cols_dropped, axis=1)
y_test = test_df['normalizzed_rtt'] 


In [5]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)


In [6]:
# split a univariate sequence into samples
from numpy import array

def split_sequence(sequence, n_steps):
    X = list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
    # gather input and output parts of the pattern
        seq_x = sequence[i:end_ix]
        X.append(seq_x)
    return array(X)

In [7]:
n_steps = 100

Xtrain_3d = split_sequence(X_train[:10000],n_steps)
Xtest_3d = split_sequence(X_test[:10000],n_steps)
Xtrain_3d.shape

(9900, 100, 58)

In [8]:
n_features = Xtrain_3d.shape[2]
Xtrain_3d  = Xtrain_3d.reshape((Xtrain_3d.shape[0], Xtrain_3d.shape[1], n_features))
Xtest_3d = Xtest_3d.reshape((Xtest_3d.shape[0], Xtest_3d.shape[1], n_features))
Xtrain_3d[0].shape

(100, 58)

In [9]:
# define model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import RMSprop

model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(n_steps, n_features),return_sequences=True))
model.add(BatchNormalization())
model.add(LSTM(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Define RMSprop optimizer with your desired learning rate
optimizer = RMSprop(learning_rate=0.0001)

# Compile your model using the RMSprop optimizer
model.compile(optimizer=optimizer, loss='mse')

In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 100, 64)           │     31,488 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization             │ (None, 100, 64)           │        256 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ lstm_1 (LSTM)                   │ (None, 64)                │     33,024 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense (Dense)                   │ (None, 32)                │      2,080 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_1 (Dense)                 │ (None, 1)                 │         33 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 66,881 (261.25 KB)

 Trainable params: 66,753 (260.75 KB)

 Non-trainable params: 128 (512.00 B)

In [14]:
#fit the model 
model.fit(Xtrain_3d,y_train[0:len(Xtrain_3d)],epochs=20, verbose=0)

#predict the model  
X_test_tensor = tf.convert_to_tensor(Xtest_3d, dtype=tf.int64) 
y_pred = model.predict(X_test_tensor, verbose=0)
y_pred

array([[0.9993285],
       [0.9993285],
       [0.9993285],
       ...,
       [0.999236 ],
       [0.999236 ],
       [0.999236 ]], dtype=float32)

In [ ]:
#loss function for LSTM
from sklearn.metrics import mean_squared_error
lstm_rmse = np.sqrt(mean_squared_error(y_test[0:len(y_pred)], y_pred))
lstm_drmse = lstm_rmse * trainrtt_std

print(f'RMSE for LSTM: ', lstm_rmse, 'Denormalied RMSE: ', lstm_drmse)

RMSE for LSTM:  0.2887142317537544 Denormalied RMSE:  3.1624252011540386
